## Blasting known HCN genes against the white clover reference genome

So far, we have not been able to differentiate homzygous dominant (e.g., AcAc) white clover individuals from heterozygous (e.g., Acac) individuals using PCR since the dominance of the markers and the multiple deletions at both loci result in preferential amplification of the smaller deletion, masking the genotype on the other homologous chromosome. We therefore need to design a PCR primer that spans the entire deletion boundary, which would allow us to resolve homozygous and heterozygous individuals on a gel based on the number and size of bands.

To facilitate this, we can blast the sequences of all known HCN genes against the recently published white clover genome (Griffiths _et al._ 2019, Plant Cell). Contigs with strong matches to genes can be retrieved and manually examined for unique sequence upstream of the deletion boundary from which we can design PCR primers.  

In [1]:
import subprocess
import os
import pandas as pd
from Bio import SeqIO

In [18]:
def exit_status(exit_code):
    """Prints success/fail of subprocess based on exit code
 
    Args:
        exit_code: ('int'): Exit code of subprocess
 
    Returns:
        None: Print to stdout.
    """
    if exit_code == 0:
        print("\nSubprocess completed successfully \n")
    else:
        print("\nThere was an error with the subprocess.\n Double check the command and try again.")

        
def makebBlastDatabase(fasta_in):
    """Creates BLAST database from fasta file.
 
    Args:
        fasta_in ('str'): Full path to input FASTA file from which database should be created
 
    Returns"
        None: Writes BLAST database to disk
    """ 
    # Name of database
    db_out = fasta_in + '.db'
  
    # Command to make BLAST database
    MakeBlastDb_Command = "makeblastdb -in " + fasta_in + " -out " + db_out + " -parse_seqids" + " -dbtype nucl"

    # Check whether subprocess successfully completed
    exit_code = subprocess.call(MakeBlastDb_Command, shell=True)
    exit_status(exit_code)
    
    
def blast_sequences(fasta_in, db, outfile):
    """BLAST's a set of sequences against BLAST database
 
    Args:
        fasta_in ('str'): Full path to input FASTA file from which database should be created
        db ('str'): Full path to input BLAST database
        outfile ('str'): Name of output file
 
    Returns"
        None: Writes BLAST results with header to disk
    """ 
    full_fasta_in_path = os.path.abspath(fasta_in)
    outpath = '/' + '/'.join(full_fasta_in_path.split('/')[1:-1]) + 'HCN_genes_BLAST/' + outfile
    
    print(full_fasta_in_path)
    print(outpath)
    print(db)
    
#     Blast_Command = "blastn -query {0} -db {1} -out {2} -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen evalue bitscore qcovs qcovhsp' -num_threads 2 -evalue 1e-10 -max_target_seqs 1 -max_hsps 1".format(fasta_in, db, outpath)
    Blast_Command = "blastn -query {0} -db {1} -out {2} -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen evalue bitscore qcovs qcovhsp' -num_threads 2 -evalue 1e-10 -max_hsps 1 -max_target_seqs 1".format(fasta_in, db, outpath)
    print(Blast_Command)
    exit_code = subprocess.call(Blast_Command, shell=True)
    exit_status(exit_code)
    
    columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'qlen', 'sstart', 'send', 'slen', 'evalue', 'bitscore', 'qcovs', 'qcovhsp']
    blast_results = pd.read_table(outpath, sep = '\t', names = columns)
    
    blast_results.to_csv(outpath, sep = '\t', index = False)
    

def retrieve_contigs(blast_results, genome_fasta, outfile):
    '''Writes FASTA with sequences of contigs that match HCN genes
    
    Args:
        blast_results ('str'): Full path to text file with results from BLAST
        genome_fasta ('str'): Full path to genome FASTA file with contigs to extract
        outfile ('str'): Name of output file
        
    Returns:
        None: Writes FASTA to disk
    '''
    
    blast_df = pd.read_table(blast_results, sep = '\t')
    
    sseqids = set(blast_df['sseqid'].tolist())
    
    genome_scaffolds = SeqIO.to_dict(SeqIO.parse(genome_fasta, 'fasta'))
    
    outpath = '/' + '/'.join(blast_results.split('/')[1:-1]) + '/' + outfile
#     print(outpath)

    with open(outpath, 'w') as f:
        
        for scaffold in sseqids:
            description = genome_scaffolds[scaffold].description
            sequence = genome_scaffolds[scaffold].seq
            
            f.write('>{0}\n{1}\n'.format(description, sequence))


### Step 1: Make BLAST database from genome

In [3]:
fasta_in = '/scratch/research/references/trifolium/repens/griffiths_2019_Plant-cell/Griffiths_et_al_genome_AllSeqs.fasta'
makebBlastDatabase(fasta_in)


Subprocess completed successfully 



### Step 2: BLAST sequences against database

In [19]:
fasta_in = '../T-repens_HCN_genes.fasta'
db = '/scratch/research/references/trifolium/repens/griffiths_2019_Plant-cell/Griffiths_et_al_genome_AllSeqs.fasta.db'
outfile = 'HCN_griffithsGenome_Blast.txt'
blast_sequences(fasta_in, db, outfile)

/scratch/research/projects/trifolium/HCN_genes_in_genome/T-repens_HCN_genes.fasta
/scratch/research/projects/trifolium/HCN_genes_in_genome/HCN_griffithsGenome_Blast.txt
/scratch/research/references/trifolium/repens/griffiths_2019_Plant-cell/Griffiths_et_al_genome_AllSeqs.fasta.db
blastn -query ../T-repens_HCN_genes.fasta -db /scratch/research/references/trifolium/repens/griffiths_2019_Plant-cell/Griffiths_et_al_genome_AllSeqs.fasta.db -out /scratch/research/projects/trifolium/HCN_genes_in_genome/HCN_griffithsGenome_Blast.txt -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen evalue bitscore qcovs qcovhsp' -num_threads 2 -evalue 1e-10 -max_hsps 1 -max_target_seqs 1

Subprocess completed successfully 



In [20]:
blast_results = pd.read_table("../HCN_griffithsGenome_Blast.txt", sep = '\t')
blast_results

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,qlen,sstart,send,slen,evalue,bitscore,qcovs,qcovhsp
0,KJ467261.1,VCDJ01010665.1,99.909,1101,1,0,1,1101,1101,19582530,19583630,54415314,0.0,2028,100,100
1,KJ467326.1,VCDJ01010665.1,97.737,1679,3,3,1,1679,1679,19570565,19572208,54415314,0.0,2857,100,100
2,JQ920491.1,VCDJ01010665.1,95.318,1132,18,5,1,1128,1128,19582530,19583630,54415314,0.0,1764,100,100
3,JQ920574.1,VCDJ01010665.1,98.889,3420,8,3,1,3392,3392,19567103,19570520,54415314,0.0,6080,100,100
4,JQ920575.1,VCDJ01010665.1,97.664,2226,13,9,1789,4014,7419,19581984,19584170,54415314,0.0,3786,30,30
5,EF990447.1,VCDJ01010675.1,100.000,2225,0,0,756,2980,3839,30228279,30230503,51265060,0.0,4109,58,58
6,JQ920572.1,VCDJ01010675.1,98.425,2920,25,6,1,2901,2901,30224409,30227326,51265060,0.0,5118,100,100
7,JQ920573.1,VCDJ01010675.1,99.604,4541,16,2,4072,8611,13521,30218214,30222753,51265060,0.0,8285,34,34
8,JQ920570.1,VCDJ01010674.1,99.515,824,4,0,1,824,824,11421697,11422520,65212296,0.0,1500,100,100
9,MH059954.1,VCDJ01010672.1,99.541,19606,62,7,59707,79297,197905,15981373,15961781,62546397,0.0,35682,10,10


### Step 3: Retrieve contigs with matches to HCN genes

In [104]:
blast_results = '/scratch/research/projects/trifolium/HCN_genome_BLAST/HCN_griffithsGenome_Blast.txt'
genome_fasta = '/scratch/research/references/trifolium/repens/griffiths_2019_Plant-cell/VCDJ01.1.fsa_nt'
outfile = 'HCN_griffithsGenome_matchedScaffolds.fasta'
retrieve_contigs(blast_results, genome_fasta, outfile)